## Carbon sources
An option at optimization of production of 7-dehydrocholesterol is analysing different carbon sources for the fermentations.

In [1]:
from cobra.io import read_sbml_model, write_sbml_model
from cobra import Reaction, Metabolite

In [2]:
model = read_sbml_model('data/yeastGEM_het.xml')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#First we can find all carbon exchange reactions:
carbon = []
carbon_names = []

for reaction in model.exchanges:
    for metabolite in reaction.reactants:
        if "C" in metabolite.elements:
            carbon.append(reaction.id)
            carbon_names.append(reaction.name) 
            
# Second, we can establish the baseline case for later reference
with model:
    max_base_growth = model.slim_optimize()
    model.objective = model.reactions.R07215
    max_base_production = model.slim_optimize()

    
# Now, we will loop through every carbon source in the model and see how good it produces 7-dehydrocholesterol.     
# We define the lists we are going to be using
max_growth = []
max_production = []
medium=model.medium

with model:
    for reaction in carbon:
        medium['r_1714'] = 0
        medium[reaction]=1
        model.medium = medium
        with model:
            max_growth.append(model.slim_optimize(error_value=0.))
            model.objective = model.reactions.R07215
            max_production.append(model.slim_optimize(error_value=0.))  # Determining max production
        medium[reaction]=0        

# We can now collect all of the data in a dataframe
production = {'Carbon': carbon_names,
             'Growth': max_growth,
             'Production': max_production}
df = pd.DataFrame(production, columns = ['Carbon', 'Growth', 'Production'])
print(df)


In [ ]:
#We remove all the carbon exchanges where growth becomes equal to 0
df= df[df['Growth'] != 0]
print(df)
# And we can create a scatterplot of the production over growth to view the distribution
plt.scatter(df.get('Growth'), df.get('Production'))
plt.xlabel('Max growth rate')
plt.ylabel('Max production')
plt.xlim(0, 0.3)
plt.ylim(0, 0.15)
plt.title('Maximal Growth rate and production')
plt.savefig("figures/carbon_sources")
plt.show()

In [ ]:
#The original growth and production of 7-dehydrocholesterol using glucose:
print(max_base_growth,max_base_production)

In [ ]:
#top 15 highest growth
df.sort_values(by='Growth', ascending=False).head(15)


In [ ]:
#Sucrose looks promising:
for reaction in model.reactions.query('sucrose exchange', 'name'):
    print(reaction.name)

In [ ]:
#Changing the carbon source to sucrose rather than glucose:
medium=model.medium
medium['r_1714'] = 0
medium["r_2058"]=1
model.medium = medium
write_sbml_model(model, "data/yeastGEM_het_suc.xml")
model_sucrose = read_sbml_model('data/yeastGEM_het_suc.xml')
print(model_sucrose.medium)

In [ ]:
model_sucrose.medium

In [ ]:
#general information:
with model_sucrose:
    model_sucrose.objective = model_sucrose.reactions.R07215
    with model_sucrose:
        model_sucrose.objective = model_sucrose.reactions.R07215
        DH_production = model.optimize().objective_value
        print("Max. 7-DH production [mmol gDW^-1 h^-1]:", DH_production)
        print("Theoretical max. yield [mmol-7-DH / mmol-sucrose]:", DH_production / (-1*model_sucrose.reactions.r_2058.flux))

These values are approximate doubles of the same values for glucose. This is due to the fact that sucrose is composed of one glucose and one fructose molecule. This is further discussed in [Report.ipynb](the report document)

Thus, it is decided that the carbon source stays as glucose, at the same availability.